## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

Применим полученные навыки и решим задачу анализа тональности отзывов. 

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать параметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор. 

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [ ]:
!pip install pymorphy2

In [ ]:
import pandas as pd
import numpy as np

import math
import nltk 
import string
import re

from sklearn.metrics import * 
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from pymorphy2 import MorphAnalyzer

from google.colab import drive
drive.mount('/content/drive/')

nltk.download('stopwords')
nltk.download('punkt')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Data/women-clothing-accessories.csv", sep="\t", usecols=[0, 1])
data.head()

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative


In [ ]:
def lemmatize(text):
  pymorphy2_analyzer = MorphAnalyzer()
  sentences = [] # Список предложений. Тут будет обработанный text
  for s in text: # Проходимся по каждой строке в тексте
    for ch in string.punctuation: # убираем всю пунктуацию из строки
      s = s.replace(ch,"")
    # приводим все слова в строке к начальной форме и нижнему регистру. Получаем normalized_tokenized_s - список лемм
    normalized_tokenized_s = [pymorphy2_analyzer.parse(word)[0].normal_form.lower() for word in word_tokenize(s)]
    sentences += [" ".join(normalized_tokenized_s)] # Соединяем обратно все леммы через пробел и записываем получившуюся строку в список предложений
  return sentences

lemm_data = data
lemm_data.review = lemmatize(data.review)
lemm_data.head()

,review,sentiment
0,качество плохой пошив ужасный горловина напере...,negative
1,товар отдать другой человек я не получить посы...,negative
2,ужасный синтетик тонкий ничего общий с предста...,negative
3,товар не прийти продавец продлить защита без м...,negative
4,кофточка голый синтетик носить не возможно,negative


In [ ]:
# Разделяем на тренировочную и тестовую выборки
x_train, x_test, y_train, y_test = train_test_split(lemm_data.review, data.sentiment, train_size = 0.7)

In [ ]:
# сначала обучаем наш векторизатор, а потом сразу применяем его к нашему набору данных"
def vectorizer_fit_transform(vectorizer):
  return vectorizer, vectorizer.fit_transform(x_train)

def bayes_classifier(vectorizer, vectorizer_x_train): # Обёртка над байесовским классификатор, чтобы получить доп. метрики
  clf = MultinomialNB() # Наивный байесовский классификатор
  clf.fit(vectorizer_x_train, y_train) # обучаем модель
  vectorizer_x_test = vectorizer.transform(x_test) # готовим тестовый векторизатор
  pred = clf.predict(vectorizer_x_test) # получаем предсказание
  # classification_report выдаст сразу несколько важных метрик типо точности, f1-score и т.д., vectorizer содержит параметры векторизатора(очевидно)
  return classification_report(y_test, pred, output_dict=True), vectorizer 

## CountVectorizer с параметром analyzer="word" (мешок n-грамм)


In [ ]:
count_vectorizer_results = [] # list tuple-ов, на первом месте которого - dict с метриками от classification_report, а на втором - параметры векторайзера
for i in range(1, 6):
  for j in range(i+1,7):
    count_vectorizer, count_vectorizer_x_train = vectorizer_fit_transform(CountVectorizer(ngram_range=(i,j), stop_words=stopwords.words("russian"), analyzer="word"))
    count_vectorizer_results += [bayes_classifier(count_vectorizer, count_vectorizer_x_train)]

In [ ]:
# Тут создадим DataFrame для CountVectorizer с параметром analyzer="word".
dict_data = []
for i, vectorizer in enumerate(count_vectorizer_results):
  if i == 0:
    continue
  metrics = vectorizer[0] # метрики от classification_report
  shape = re.findall('ngram_range=\([0-9]+, [0-9]+\)', str(vectorizer[1]))[0][12::] # достаём ngram_range регулярками
  dict_data += [{'Vectorizer' : 'CountVectorizer', 'Analyzer type' : 'word', 'ngram_range' : shape, 'Accuracy' : metrics['accuracy'], 'WA Precision' : metrics['weighted avg']['precision'], 'WA f1-score' : metrics['weighted avg']['f1-score'], 'WA recall' : metrics['weighted avg']['recall']}]
count_vectorizer_df = pd.DataFrame(dict_data)
count_vectorizer_df.sort_values(by=['Accuracy'], ascending=False).head() # Сортируем по метрике Accuracy

,Vectorizer,Analyzer type,ngram_range,Accuracy,WA Precision,WA f1-score,WA recall
0,CountVectorizer,word,"(1, 3)",0.714048,0.713459,0.713640,0.714048
1,CountVectorizer,word,"(1, 4)",0.713899,0.713000,0.713383,0.713899
2,CountVectorizer,word,"(1, 5)",0.713825,0.712814,0.713266,0.713825
3,CountVectorizer,word,"(1, 6)",0.713677,0.712611,0.713096,0.713677
4,CountVectorizer,word,"(2, 3)",0.667272,0.663833,0.663591,0.667272


## CountVectorizer с параметром analyzer="char" (символьные n-граммы)

In [ ]:
# Всё то жесамое как и для мешка n-грамм, просто поменяли один параметр
char_count_vectorizer_results = []
for i in range(1,6):
  for j in range(i+1,7):
    count_vectorizer, count_vectorizer_x_train = vectorizer_fit_transform(CountVectorizer(ngram_range=(i,j), stop_words=stopwords.words("russian"), analyzer="char"))
    char_count_vectorizer_results += [bayes_classifier(count_vectorizer, count_vectorizer_x_train)]

In [ ]:
# Строим DataFrame
dict_data = []
for i, vectorizer in enumerate(char_count_vectorizer_results):
  metrics = vectorizer[0]
  shape = shape = re.findall('ngram_range=\([0-9]+, [0-9]+\)', str(vectorizer[1]))[0][12::]
  dict_data += [{'Vectorizer' : 'CountVectorizer', 'Analyzer type' : 'char', 'ngram_range' : shape, 'Accuracy' : metrics['accuracy'], 'WA Precision' : metrics['weighted avg']['precision'], 'WA f1-score' : metrics['weighted avg']['f1-score'], 'WA recall' : metrics['weighted avg']['recall']}]
char_count_vectorizer_df = pd.DataFrame(dict_data)
# char_count_vectorizer_df2.sort_values(by=['Accuracy'], ascending=False).head(10)

# char_count_vectorizer_df = pd.concat([char_count_vectorizer_df1, char_count_vectorizer_df2], ignore_index=True)
char_count_vectorizer_df.sort_values(by=['Accuracy'], ascending=False).head()


,Vectorizer,Analyzer type,ngram_range,Accuracy,WA Precision,WA f1-score,WA recall
14,CountVectorizer,char,"(5, 6)",0.719122,0.729193,0.721529,0.719122
13,CountVectorizer,char,"(4, 6)",0.716862,0.727602,0.719312,0.716862
11,CountVectorizer,char,"(3, 6)",0.713974,0.725750,0.716537,0.713974
8,CountVectorizer,char,"(2, 6)",0.711566,0.723718,0.714185,0.711566
12,CountVectorizer,char,"(4, 5)",0.711307,0.722533,0.713699,0.711307


## TfidfVectorizer

In [ ]:
''' max_df - при построении словаря игнорировать слова, чья частота появления в тексте строго больше заданного параметра.
    min_df - при построении словаря игнорировать слова, чья частота появления в тексте строго меньше заданного параметра.
    max_features - если этот параметр не равен None, то строим словарь учитывая только топ max_features слов, упорядоченных(по убыванию) по частоте появления в тексте.
'''
import warnings
warnings.filterwarnings('ignore')

tfidf_vectorizer_results = []
for i in range(1, 3):
  for j in range(i + 1, 4):
    for max_df in (0.7, 0.1):
      for min_df in (0.01, 0.02):
        for max_features in (2048, 4096):
          tfidf_vectorizer, tfidf_vectorizer_x_train = vectorizer_fit_transform(TfidfVectorizer(ngram_range=(i, j), max_df=max_df, min_df=min_df, max_features=max_features))
          tfidf_vectorizer_results += [bayes_classifier(tfidf_vectorizer, tfidf_vectorizer_x_train)]

In [ ]:
# Строим DataFrame
dict_data = []
for vectorizer in tfidf_vectorizer_results:
  metrics = vectorizer[0]
  max_df = re.findall('max_df=0[.0-9]*', str(vectorizer[1]))[0][7::]
  max_features = re.findall('max_features=[0-9]+', str(vectorizer[1]))[0][13::]
  min_df = re.findall('min_df=0[.0-9]*', str(vectorizer[1]))[0][7::]
  shape = re.findall('ngram_range=\([0-9]+, [0-9]+\)', str(vectorizer[1]))[0][12::]
  dict_data += [{'Vectorizer' : 'TfidfVectorizer', 'Analyzer type' : 'word', 'max_df' : max_df, 'min_df' : min_df, 'max_features' : max_features, 'ngram_range' : shape, 'Accuracy' : metrics['accuracy'], 'WA Precision' : metrics['weighted avg']['precision'], 'WA f1-score' : metrics['weighted avg']['f1-score'], 'WA recall' : metrics['weighted avg']['recall']}]
tfidf_vectorizer_df = pd.DataFrame(dict_data).sort_values(by=['Accuracy'], ascending=False)
tfidf_vectorizer_df.sort_values(by=['Accuracy'], ascending=False).head(10)

,Vectorizer,Analyzer type,max_df,min_df,max_features,ngram_range,Accuracy,WA Precision,WA f1-score,WA recall
0,TfidfVectorizer,word,0.7,0.01,2048,"(1, 2)",0.679530,0.687049,0.681478,0.679530
1,TfidfVectorizer,word,0.7,0.01,4096,"(1, 2)",0.679530,0.687049,0.681478,0.679530
8,TfidfVectorizer,word,0.7,0.01,2048,"(1, 3)",0.678716,0.686861,0.680524,0.678716
9,TfidfVectorizer,word,0.7,0.01,4096,"(1, 3)",0.678716,0.686861,0.680524,0.678716
4,TfidfVectorizer,word,0.1,0.01,2048,"(1, 2)",0.669716,0.675661,0.671711,0.669716
5,TfidfVectorizer,word,0.1,0.01,4096,"(1, 2)",0.669716,0.675661,0.671711,0.669716
13,TfidfVectorizer,word,0.1,0.01,4096,"(1, 3)",0.669383,0.675840,0.671270,0.669383
12,TfidfVectorizer,word,0.1,0.01,2048,"(1, 3)",0.669383,0.675840,0.671270,0.669383
10,TfidfVectorizer,word,0.7,0.02,2048,"(1, 3)",0.653124,0.658615,0.654325,0.653124
11,TfidfVectorizer,word,0.7,0.02,4096,"(1, 3)",0.653124,0.658615,0.654325,0.653124


## Итог

In [ ]:
# tfidf_vectorizer_df.drop(['max_df', 'min_df', 'max_features'], axis=1, inplace=True) # Убираем лишние столбцы из DataFrame, чтобы соединить всё в один

vectorizer_dfs = pd.concat([count_vectorizer_df, char_count_vectorizer_df, tfidf_vectorizer_df], ignore_index=True)
vectorizer_dfs.sort_values(by=['Accuracy'], ascending=False).head(18)

,Vectorizer,Analyzer type,ngram_range,Accuracy,WA Precision,WA f1-score,WA recall
28,CountVectorizer,char,"(5, 6)",0.719122,0.729193,0.721529,0.719122
27,CountVectorizer,char,"(4, 6)",0.716862,0.727602,0.719312,0.716862
0,CountVectorizer,word,"(1, 3)",0.714048,0.713459,0.713640,0.714048
25,CountVectorizer,char,"(3, 6)",0.713974,0.725750,0.716537,0.713974
1,CountVectorizer,word,"(1, 4)",0.713899,0.713000,0.713383,0.713899
2,CountVectorizer,word,"(1, 5)",0.713825,0.712814,0.713266,0.713825
3,CountVectorizer,word,"(1, 6)",0.713677,0.712611,0.713096,0.713677
22,CountVectorizer,char,"(2, 6)",0.711566,0.723718,0.714185,0.711566
26,CountVectorizer,char,"(4, 5)",0.711307,0.722533,0.713699,0.711307
18,CountVectorizer,char,"(1, 6)",0.710937,0.723146,0.713541,0.710937


## Вывод
Лучше всего себя показали векторайзеры CountVectorizer с параметром analyzer="char". Из них самый лучший с параметром ngram_range = (5, 6). На втором месте векторайзеры CountVectorizer с параметром analyzer="word". Из них лучший с параметром ngram_range = (1, 3) оказался на 3-м месте в общем датафрейме. Векторайзеры TfidfVectorizer дали худший результат. Лучший из них оказался лишь на 18 месте с параметром ngram_range = (1, 2) *(полные параметры можно увидеть в датафрейме про TfidfVectorizer)*

## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах. 

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [ ]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **?а** - ноль или один символ **а**
* **+а** - один или более символов **а**
* **\*а** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа

Пример:
Выражению \*a?b. соответствуют последовательности a, ab, abc, aa, aac НО НЕ abb!

Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**: 
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc') 
print(result)

Вопрос на внимательность: почему нет abcx?

Ответ: findall возвращает список всех найденных непересекающихся совпадений. "abca" и "abcx" пересекаются - у них 'а' общий.

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [ ]:
sentence1 = "There's a lady who's sure all that glitters is gold"
first_two_letters = [x[:2:] for x in re.findall('[A-Za-z\'\-]+[ .!?\n]?', sentence1)]
first_two_letters

### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie') 
print(result)

можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2) 
print(result)

**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [ ]:
text = "That. she. wore. for. the. first. time. today."

re.split('\.', text, maxsplit=2)

### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print (result)

**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [ ]:
text = 'You load 16 tons, what do you get?'
result = re.sub('[0-9]', 'DIG', text)
result

**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [ ]:
string = 'web-site: https://ru.wikipedia.org/wiki/%D0%97%D0%B0%D0%B3%D0%BB%D0%B0%D0%B2%D0%BD%D0%B0%D1%8F_%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B8%D1%86%D0%B0'
result = re.sub('https://.+', '', string)
result

### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [ ]:
string = """
Money, it's a crime
Share it fairly but don't take a slice of my pie
Money, so they say
Is the root of all evil today
"""
prog = re.compile('[0-9A-Za-z,.!?\-\']{3}[0-9A-Za-z,.!?\-\']+')
prog.findall(string)

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [ ]:
text = "abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz"
prog = re.compile('.+@gmail.com')
prog.findall(text)